# 1. Cache

__Installing dependencies__

In [1]:
!python3 -m pip install -q dff[tutorials]
# Installs dff with dependencies for running tutorials
# To install the minimal version of dff, use `pip install dff`
# To install other options of dff, use `pip install dff[OPTION_NAME1,OPTION_NAME2]`
# where OPTION_NAME can be one of the options from EXTRA_DEPENDENCIES.
# e.g `pip install dff[ydb, mysql]` installs dff with dependencies for using Yandex Database and MySQL
# EXTRA_DEPENDENCIES can be found in
# https://github.com/deeppavlov/dialog_flow_framework/blob/dev/setup.py#L155

__Running tutorial__

In [2]:
from dff.script.conditions import true
from dff.script import Context, TRANSITIONS, RESPONSE, Message
from dff.script.labels import repeat
from dff.pipeline import Pipeline
from dff.utils.turn_caching import cache
from dff.utils.testing.common import (
    check_happy_path,
    is_interactive_mode,
    run_interactive_mode,
)


external_data = {"counter": 0}

In [3]:
@cache
def cached_response(_):
    """
    This function execution result will be saved
    for any set of given argument(s).
    If the function will be called again
    with the same arguments it will prevent it from execution.
    The cached values will be used instead.
    The cache is stored in a library-wide singleton,
    that is cleared in the end of execution of actor and/or pipeline.
    """
    external_data["counter"] += 1
    return external_data["counter"]


def response(ctx: Context, _, *__, **___) -> Message:
    if ctx.validation:
        return Message()
    return Message(
        text=f"{cached_response(1)}-{cached_response(2)}-"
        f"{cached_response(1)}-{cached_response(2)}"
    )

In [4]:
toy_script = {"flow": {"node1": {TRANSITIONS: {repeat(): true()}, RESPONSE: response}}}

happy_path = (
    (Message(), Message(text="1-2-1-2")),
    (Message(), Message(text="3-4-3-4")),
    (Message(), Message(text="5-6-5-6")),
)

pipeline = Pipeline.from_script(toy_script, start_label=("flow", "node1"))

In [5]:
if __name__ == "__main__":
    check_happy_path(pipeline, happy_path)
    if is_interactive_mode():
        run_interactive_mode(pipeline)

(user) >>> 
 (bot) <<< text='1-2-1-2'
(user) >>> 
 (bot) <<< text='3-4-3-4'
(user) >>> 
 (bot) <<< text='5-6-5-6'
